In [ ]:
import os
import torch

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F
from google.colab import drive
from PIL import Image
import matplotlib.pyplot as plt
import sys
import pandas as pd
from collections import defaultdict
import json
import random
from torch.utils.data import Dataset
import numpy as np
import torchvision.models.detection as detection
from torch.utils.data import DataLoader
import torch.utils
import torch
from PIL import Image as read_image
from torchvision.io import read_image
from torchvision.transforms import v2
from transforms import RandomPhotometricDistort, RandomZoomOut, RandomIoUCrop, RandomHorizontalFlip,RandomVerticalFlip ,ToTensor, Compose
from torchvision.transforms.functional import convert_image_dtype

In [ ]:
drive.mount('/content/drive')
path_base = '/content/drive/MyDrive/Academico/TD8 personal '
sys.path.append(path_base)

Mounted at /content/drive


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:

train_file = path_base + '/classificationDownload (1)/train.csv'
test_file = path_base + '/classificationDownload (1)/test.csv'
classification_file = path_base + '/classificationDownload (1)/classifications.csv'

train_images = pd.read_csv(train_file)['image_filename'].tolist()
test_images = pd.read_csv(test_file)['image_filename'].tolist()

classification_data = pd.read_csv(classification_file)

train_data = classification_data[classification_data['image_filename'].isin(train_images)]
test_data = classification_data[classification_data['image_filename'].isin(test_images)]


def get_transform(train=True):
    transforms = []
    if train:
        transforms.append(RandomPhotometricDistort())
        transforms.append(RandomZoomOut(fill=list((207., 202., 205.))))
        transforms.append(RandomHorizontalFlip(p=0.5))
        transforms.append(RandomVerticalFlip(p=0.5))

    transforms.append(v2.ToDtype(torch.float, scale=True))
    return Compose(transforms)


def group_detections(data):
    grouped_data = {}
    for _, row in data.iterrows():
        image_name = row['image_filename']
        if image_name not in grouped_data:
            grouped_data[image_name] = {
                'image_filename': image_name,
                'image_id': row['image_id'],
                'image_doi': row['image_doi'],
                'classifications': []
            }

        grouped_data[image_name]['classifications'].append({
            'nucleus_x': row['nucleus_x'],
            'nucleus_y': row['nucleus_y'],
            'bethesda_system': row['bethesda_system'],
            'cell_id': row['cell_id']
        })
    return list(grouped_data.values())


train_data_grouped = group_detections(train_data)
test_data_grouped = group_detections(test_data)


train_counts = defaultdict(int)
test_counts = defaultdict(int)


for item in train_data_grouped:
    for classification in item['classifications']:
        train_counts[classification['bethesda_system']] += 1


for item in test_data_grouped:
    for classification in item['classifications']:
        test_counts[classification['bethesda_system']] += 1


class PAPDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

        self.label_map = {
            "SCC": 1,
            "HSIL": 2,
            "LSIL": 3,
            "ASC-H": 4,
            "ASC-US": 5,
            "Negative for intraepithelial lesion": 6
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = read_image(path_base + '/Imagenes/' + item['image_filename'])

        image_doi = item['image_doi']
        image_id = item['image_id']
        boxes = []
        labels = []
        for classification in item['classifications']:
            nucleus_x = classification['nucleus_x']
            nucleus_y = classification['nucleus_y']
            x_min = nucleus_x - 50
            y_min = nucleus_y - 50
            x_max = nucleus_x + 50
            y_max = nucleus_y + 50
            boxes.append([x_min, y_min, x_max, y_max])
            labels.append(self.label_map[classification['bethesda_system']])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)

        labels = torch.as_tensor(labels, dtype=torch.int64)
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        target = {
            "boxes": boxes,
            "labels": labels,
            "area": area,
            "image_id": image_id
        }

        num_objs = boxes.shape[0]
        target["iscrowd"] = torch.zeros((num_objs,), dtype=torch.int64)

        if self.transform:
            img, target = self.transform(img, target)
        return img, target


train_dataset = PAPDataset(train_data_grouped, transform=get_transform(train=True))
test_dataset = PAPDataset(test_data_grouped, transform=get_transform(train=False))

print("Training Counts:", dict(train_counts))
print("Test Counts:", dict(test_counts))
print(f"Training Dataset Size: {len(train_dataset)}")
print(f"Test Dataset Size: {len(test_dataset)}")


Training Counts: {'SCC': 145, 'Negative for intraepithelial lesion': 6043, 'LSIL': 1221, 'HSIL': 1588, 'ASC-H': 808, 'ASC-US': 550}
Test Counts: {'ASC-H': 117, 'Negative for intraepithelial lesion': 736, 'ASC-US': 56, 'LSIL': 139, 'SCC': 16, 'HSIL': 115}
Training Dataset Size: 360
Test Dataset Size: 40


In [ ]:

from ssd import  ssd512_resnet50_Upgraded_version2
model = ssd512_resnet50_Upgraded_version2(load = True)
model = model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/content/drive/MyDrive/Academico/TD8 personal /ssd.py:1397: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. 

Se cargaron los pesos de SipakMed en Resnet 50


In [ ]:
from ssd import SSDClassificationHead, SSDRegressionHead
num_classes = 6+1  # Ejemplo con 6  clases + 1 clase background


in_channels = [2048, 512, 512, 256, 256, 256]
num_anchors = [4, 6, 6, 6, 4, 4]
model.head.classification_head = SSDClassificationHead(in_channels, num_anchors, num_classes)




In [ ]:

def collate_fn(batch):
    return tuple(zip(*batch))



batch_size = 16

train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.SequentialSampler(test_dataset)

train_batch_sampler = train_batch_sampler = torch.utils.data.BatchSampler(
            train_sampler, batch_size, drop_last=True)


data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_sampler=train_batch_sampler,
        collate_fn=collate_fn)


data_loader_test = torch.utils.data.DataLoader(
        test_dataset, batch_size=1,
        sampler=test_sampler,
        collate_fn=collate_fn)

In [ ]:
import math
from utils import MetricLogger, warmup_lr_scheduler, SmoothedValue
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    metric_logger = MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) if not isinstance(v, int) else v for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        print(loss_value)

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            break

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger



'\nMetricLogger te ayuda a monitorear métricas a lo largo del entrenamiento.\nWarmup Learning Rate Scheduler te permite empezar el entrenamiento de manera más suave ajustando gradualmente la tasa de aprendizaje durante las primeras iteraciones para evitar inestabilidad.\n'

In [ ]:
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py
!pip install -U pycocotools

--2024-11-16 13:45:15--  https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4063 (4.0K) [text/plain]
Saving to: ‘engine.py.1’

engine.py.1         100%[===================>]   3.97K  --.-KB/s    in 0s      

2024-11-16 13:45:16 (77.4 MB/s) - ‘engine.py.1’ saved [4063/4063]

--2024-11-16 13:45:16--  https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8388 (8.2K) [text/plai

In [ ]:
from torchvision.datasets import VOCDetection
from torchvision.transforms import Compose, ToTensor, RandomHorizontalFlip, Normalize
from engine import evaluate
import time
import datetime


params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
        params, lr = 0.003, momentum=0.9, weight_decay = 0.0005)

lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80,110], gamma=0.1)

model.to(device)
best_ap = 0.0

print("Start training")
start_time = time.time()
for epoch in range(50):
  train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
  lr_scheduler.step()
  ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
  if ap > best_ap:
    best_ap = ap
    best_model_weights = model.state_dict()

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))








Start training
47.544883728027344
Epoch: [0]  [ 0/22]  eta: 0:00:55  lr: 0.000146  time: 2.5254  data: 2.0614  max mem: 11119
44.46854782104492
37.542789459228516
33.11880874633789
34.88504409790039
23.957368850708008
20.475515365600586
16.36014175415039
12.639932632446289
14.155560493469238
13.40781307220459
11.160442352294922
12.97153091430664
10.629087448120117
10.049091339111328
9.175657272338867
6.897123336791992
12.77280044555664
19.85576629638672
17.26740837097168
9.346115112304688
Epoch: [0]  [20/22]  eta: 0:00:04  lr: 0.003000  time: 2.2585  data: 1.9171  max mem: 12134
23.667875289916992
Epoch: [0]  [21/22]  eta: 0:00:02  lr: 0.003000  time: 2.2219  data: 1.8749  max mem: 12134
Epoch: [0] Total time: 0:00:49 (2.2709 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:03  model_time: 0.0168 (0.0168)  evaluator_time: 0.0062 (0.0062)  time: 0.0880  data: 0.0614  max mem: 12134
Test:  [39/40]  eta: 0:00:00  model_time: 0.0161 (0.0165)  evaluator_time: 0.0074 (0.010

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/upgraded_sdd512_resnet_SipakMedAug_version2.pth" )

In [ ]:

start_time = time.time()


for epoch in range(50, 100):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))



2.2943098545074463
Epoch: [50]  [ 0/22]  eta: 0:00:44  lr: 0.003000  time: 2.0250  data: 1.7975  max mem: 12562
2.695157527923584
2.1742944717407227
2.726614236831665
2.270151138305664
2.1155037879943848
2.8327698707580566
2.7034499645233154
2.299151659011841
2.7961320877075195
2.2763566970825195
2.653653144836426
2.2350292205810547
2.196681499481201
2.8620240688323975
2.771052598953247
2.915137767791748
2.6096572875976562
2.6747894287109375
2.1161460876464844
2.6697793006896973
Epoch: [50]  [20/22]  eta: 0:00:04  lr: 0.003000  time: 2.1860  data: 1.8519  max mem: 12562
2.322240114212036
Epoch: [50]  [21/22]  eta: 0:00:02  lr: 0.003000  time: 2.1861  data: 1.8533  max mem: 12562
Epoch: [50] Total time: 0:00:48 (2.1918 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:04  model_time: 0.0390 (0.0390)  evaluator_time: 0.0197 (0.0197)  time: 0.1238  data: 0.0614  max mem: 12562
Test:  [39/40]  eta: 0:00:00  model_time: 0.0163 (0.0172)  evaluator_time: 0.0206 (0.0307)  time

In [ ]:
start_time = time.time()


for epoch in range(100, 120):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
    if ap > best_ap:
      best_ap = ap
      best_model_weights = model.state_dict()

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

1.70718252658844
Epoch: [100]  [ 0/22]  eta: 0:00:58  lr: 0.000300  time: 2.6460  data: 2.3670  max mem: 12571
2.153564929962158
1.9086477756500244
1.7757322788238525
2.0496490001678467
1.9691163301467896
2.2097957134246826
2.047030210494995
1.904306173324585
1.981797695159912
2.239105701446533
2.2780954837799072
2.0369420051574707
1.8162360191345215
1.7973543405532837
1.799560785293579
2.1044459342956543
1.7037020921707153
2.213313102722168
1.8511998653411865
1.861530065536499
Epoch: [100]  [20/22]  eta: 0:00:04  lr: 0.000300  time: 2.1335  data: 1.7875  max mem: 12571
2.273467779159546
Epoch: [100]  [21/22]  eta: 0:00:02  lr: 0.000300  time: 2.1132  data: 1.7707  max mem: 12571
Epoch: [100] Total time: 0:00:47 (2.1545 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:03  model_time: 0.0170 (0.0170)  evaluator_time: 0.0125 (0.0125)  time: 0.0962  data: 0.0630  max mem: 12571
Test:  [39/40]  eta: 0:00:00  model_time: 0.0163 (0.0167)  evaluator_time: 0.0196 (0.0267)  ti

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/upgraded_sdd512_resnet_SipakMedAug_version2.pth" )

El AP del mejor modelo siendo esre el que guardamos es:
0.3069459722170868


In [ ]:
start_time = time.time()


for epoch in range(120, 150):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
    if ap > best_ap:
      best_ap = ap
      best_model_weights = model.state_dict()

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

1.8800132274627686
Epoch: [120]  [ 0/22]  eta: 0:00:39  lr: 0.000030  time: 1.7910  data: 1.4703  max mem: 12571
2.316411256790161
1.9333527088165283
1.8758898973464966
2.0680134296417236
2.1634318828582764
1.7454885244369507
1.7861878871917725
2.0043222904205322
2.1134283542633057
2.102367639541626
1.7995266914367676
1.9045196771621704
1.853518009185791
1.7117716073989868
2.08345103263855
1.890058994293213
1.6713311672210693
1.7225052118301392
1.8411095142364502
2.4233789443969727
Epoch: [120]  [20/22]  eta: 0:00:04  lr: 0.000030  time: 2.1301  data: 1.8014  max mem: 12571
1.8613125085830688
Epoch: [120]  [21/22]  eta: 0:00:02  lr: 0.000030  time: 2.1033  data: 1.7769  max mem: 12571
Epoch: [120] Total time: 0:00:46 (2.1133 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:03  model_time: 0.0171 (0.0171)  evaluator_time: 0.0128 (0.0128)  time: 0.0974  data: 0.0638  max mem: 12571
Test:  [39/40]  eta: 0:00:00  model_time: 0.0162 (0.0164)  evaluator_time: 0.0208 (0.0264

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/upgraded_sdd512_resnet_SipakMedAug_version2.pth" )

El AP del mejor modelo siendo esre el que guardamos es:
0.3120155802995794
